In [1]:
import os, sys
import csv
import json
import pandas as pd

In [2]:
csv_file = 'Run3Summer22.csv'
campaign = 'Run3Summer22'

In [3]:
def dict_to_dataframe(data_dict):
    data_list = []
    for sample, subsamples in data_dict.items():
        for subsample, lumi in subsamples.items():
            data_list.append([sample, subsample, lumi])    
    df = pd.DataFrame(data_list, columns=["sample", "subsample", "lumi"])
    return df

def write_lumidata_into_json(df, campaign):
    outdir='extracted_jsons'
    os.makedirs(outdir, exist_ok=True)
    outfile = os.path.join(outdir, f'lumidata_{campaign}.json')
    with open(outfile, 'w') as json_file: json.dump(df, json_file, indent=4)
    print('File written: ' + outfile)
    
print('Functions loaded.')

Functions loaded.


In [4]:
df_lumi = pd.DataFrame()
with open(csv_file, newline='') as csvfile:
    reader = csv.reader(csvfile)
    lumilist = list(reader)

df_lumi = pd.DataFrame(lumilist)
df_lumi=df_lumi[[1, 8]]
df_lumi = df_lumi[~df_lumi.apply(lambda row: all(cell == '' for cell in row), axis=1)].reset_index(drop=True) # Dropping empty rows.

# Dropping rows where the first column has a value (e.g., process name) but all other columns are empty
df_lumi = df_lumi[~df_lumi.apply(lambda row: row.iloc[0] != '' and all(cell == '' for cell in row.iloc[1:]), axis=1)].reset_index(drop=True)

# Remove rows containing 'sample_subsample'
df_lumi = df_lumi[df_lumi.iloc[:, 0] != 'sample_subsample'].reset_index(drop=True)

df_lumi.columns = ['sample_subsample', 'lumi']
df_lumi['lumi'] = df_lumi['lumi'].astype(float)

split_values = df_lumi['sample_subsample'].str.split('_', n=1)
df_lumi['sample'] = split_values.str[0]
df_lumi['subsample'] = split_values.str[1]

df_lumi = df_lumi[['sample', 'subsample', 'lumi']]
display(df_lumi)

,sample,subsample,lumi
0,WtoLNu,Inclusive,1.314158e+03
1,WGtoLNuG,10to100,7.631562e+04
2,WGtoLNuG,100to200,4.451589e+05
3,WGtoLNuG,200to400,1.665461e+06
4,WGtoLNuG,400to600,2.079523e+07
...,...,...,...
90,QCDMu,300to470,4.325455e+04
91,QCDMu,470to600,2.741732e+05
92,QCDMu,600to800,9.330059e+05
93,QCDMu,800to1000,1.008112e+07


In [5]:
datadict = {}
if campaign == 'Run3Summer22':
    datadict = {"Muon":{"C":7980.4,"D":7980.4},"EGamma":{"C":7980.4,"D":7980.4}}

signaldict = {
    "VLLS_ele":{
        "M100":     512598.29,
        "M125":    1091218.35,
        "M150":    2064227.59,
        "M200":    5696971.43,
        "M250":   10804152.17,
        "M300":   21769257.64,
        "M350":   15935520.00,
        "M400":   27645555.56,
        "M450":   44700229.36,
        "M500":   72181021.90,
        "M750":  254269230.77,
        "M1000":1268884615.38
    },
    "VLLS_mu":{
        "M100":     508329.91,
        "M125":    1100348.62,
        "M150":    2064344.83,
        "M200":    5710047.62,
        "M250":   10794347.83,
        "M300":   21809519.65,
        "M400":   27595277.78,
        "M450":   45337844.04,
        "M500":   72847810.22,
        "M750":  255569230.77,
        "M1000":1275589743.59
    },
    "VLLD_ele":{
        "M100":     6560.41,
        "M200":    54213.24,
        "M300":    85061.86,
        "M400":   270022.05,
        "M600":  1651744.97,
        "M800":  6824207.49,
        "M1000":25011328.53
    },
    "VLLD_mu":{
        "M100":   6622.84,
        "M200":  54344.12,
        "M300":  85986.25,
        "M400": 267905.18,
        "M600":1670469.80,
        "M800":7136311.24
    }
}

df_lumi     = df_lumi.sort_values(by=["sample", "subsample"]).reset_index(drop=True)
df_signal   = dict_to_dataframe(signaldict)
df_data     = dict_to_dataframe(datadict)
df_campaign = pd.concat([df_lumi, df_signal, df_data])

In [6]:
df_campaign

,sample,subsample,lumi
0,DYto2L,10to50,2321.196
1,DYto2L,4Jets10to50,8848.936
2,DYto2L,4Jets50toInf,13603.517
3,DYto2L,50toInf,15350.715
4,Higgs,GGHHto2B,123112.583
...,...,...,...
35,VLLD_mu,M800,7136311.240
0,Muon,C,7980.400
1,Muon,D,7980.400
2,EGamma,C,7980.400


In [7]:
lumidata = {}
for sample, subsample, lumi in df_campaign.itertuples(index=False):
    if sample not in lumidata: lumidata[sample] = {}
    lumidata[sample][subsample] = lumi

write_lumidata_into_json(lumidata, campaign)

File written: extracted_jsons/lumidata_Run3Summer22.json
